In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate

from sklearn.metrics import confusion_matrix, accuracy_score

<b> Get data for:</b>
- TeamID key
- Regular season game results (2002/2003 - 2017/2018)
- NCAA Tournament game results (2002/2003 - 2017/2018)

In [2]:
# DF with key linking TeamName and TeamID
teams = pd.read_csv('Teams.csv')
teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2018
1,1102,Air Force,1985,2018
2,1103,Akron,1985,2018
3,1104,Alabama,1985,2018
4,1105,Alabama A&M,2000,2018


In [3]:
# Detailed results from all March Madness games since 2002-2003 season
# Each row is a game with stats for winning and losing team
ncaa_results = pd.read_csv('NCAATourneyDetailedResults.csv')
ncaa_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


In [4]:
# Detailed results from all regular season games since 2002-2003 season
# Each row is a game with stats for winning and losing team
regular_season_results = pd.read_csv('2018/RegularSeasonDetailedResults.csv')
regular_season_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


### Regular Season Summary Statistics

<b> Aggregate results from individual games into season-long summary statistics for each team for each season </b>

Extract statistics for each team. Initally, each row in DF is a game. Convert so that there is a row for each team for each game. <br>
I.e. a single game now comprises two rows, one for each team

In [5]:
# Format column names - remove L and W prefixes
base_colNames = ('TeamID', 'Score', 'TeamID', 'Score','FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR',
       'Ast', 'TO', 'Stl', 'Blk', 'PF')

W_colNames = tuple(['W'+x for x in base_colNames])
L_colNames = tuple(['L'+x for x in base_colNames])
Opp_colNames = tuple(['Opp'+x for x in base_colNames])

In [6]:
# Create subset of regular season results for winners
# Winning team = stats, losing team = Opponent Stats
stats_agg_w = regular_season_results.copy()
stats_agg_w.rename(columns={i:j for i,j in zip(W_colNames,base_colNames)}, inplace=True)
stats_agg_w.rename(columns={i:j for i,j in zip(L_colNames,Opp_colNames)}, inplace=True)

stats_agg_w.head()

,Season,DayNum,TeamID,Score,OppTeamID,OppScore,WLoc,NumOT,FGM,FGA,...,OppFGA3,OppFTM,OppFTA,OppOR,OppDR,OppAst,OppTO,OppStl,OppBlk,OppPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [7]:
# Subset of data for losses
# Losing team = stats, winnint team = Opponent Stats
stats_agg_l = regular_season_results.copy()
stats_agg_l.rename(columns={i:j for i,j in zip(W_colNames,Opp_colNames)}, inplace=True)
stats_agg_l.rename(columns={i:j for i,j in zip(L_colNames,base_colNames)}, inplace=True)
stats_agg_l.head()

,Season,DayNum,OppTeamID,OppScore,TeamID,Score,WLoc,NumOT,OppFGM,OppFGA,...,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [8]:
# Combine winner and loser subset: result table with all game results for all teams
stats_agg = pd.concat([stats_agg_w, stats_agg_l], axis=0)

<b> Aggregate individual game statistcs to season long summary stats for each team for each season </b>

In [9]:
# Aggregate individual game statistics; DF indicies = Season, Team
my_stats_avg = stats_agg.groupby(['Season', 'TeamID']).mean()
my_stats_avg = my_stats_avg.drop(['DayNum', 'OppTeamID', 'NumOT'], axis=1)

In [10]:
# Calculate Min/Max stats
max_stats = stats_agg[['Season', 'TeamID', 'Score', 'OppScore', 'TO', 'OppTO', 'FGA3', 'FGA']].groupby(['Season', 'TeamID']).max()
min_stats = stats_agg[['Season', 'TeamID', 'Score', 'OppScore', 'TO', 'OppTO', 'FGA3', 'FGA']].groupby(['Season', 'TeamID']).min()

# Append prefix to column names
max_stats.columns = ['max_' + str(col) for col in max_stats.columns]
min_stats.columns = ['min_' + str(col) for col in min_stats.columns]

In [11]:
# Combine max, min, avg stats
all_stats = pd.concat([my_stats_avg, max_stats, min_stats], axis=1)

In [12]:
# Add percentage features
all_stats['FTpct'] = all_stats['FTM'] / all_stats['FTA']
all_stats['FG3pct'] = all_stats['FGM3'] / all_stats['FGA3']
all_stats['FGpct'] = all_stats['FGM'] / all_stats['FGA']

In [13]:
# Write stats df to pickle
all_stats.to_pickle("all_stats.pkl")

all_stats.head()

Ast       Blk         DR        FGA       FGA3  \
Season TeamID                                                         
2003   1102    13.000000  1.785714  16.821429  39.785714  20.821429   
       1103    15.222222  2.333333  19.925926  55.851852  16.074074   
       1104    12.107143  3.785714  23.928571  57.178571  19.857143   
       1105    14.538462  2.076923  23.115385  61.615385  20.769231   
       1106    11.678571  3.142857  23.857143  55.285714  17.642857   

                     FGM      FGM3        FTA        FTM         OR    ...     \
Season TeamID                                                          ...      
2003   1102    19.142857  7.821429  17.107143  11.142857   4.178571    ...      
       1103    27.148148  5.444444  25.851852  19.037037   9.777778    ...      
       1104    24.035714  6.357143  20.928571  14.857143  13.571429    ...      
       1105    24.384615  7.576923  21.846154  15.423077  13.500000    ...      
       1106    23.428571  6.107143  16.464286  10.642857  12.285714    ...      

               max_FGA  min_Score  min_OppScore  min_TO  min_OppTO  min_FGA3  \
Season TeamID                                                                  
2003   1102         51         33            33       5          6         9   
       1103         75         52            55       6          9         7   
       1104         73         46            48       7          7        11   
       1105         73         40            53       7         10        10   
       1106         71         43            33       8          7        10   

               min_FGA     FTpct    FG3pct     FGpct  
Season TeamID                                         
2003   1102         32  0.651357  0.375643  0.481149  
       1103         36  0.736390  0.338710  0.486074  
       1104         42  0.709898  0.320144  0.420362  
       1105         45  0.705986  0.364815  0.395755  
       1106         43  0.646421  0.346154  0.423773  

[5 rows x 43 columns]

### Combine Historical Tournament Results with Regular Season Aggregated stats

<b> Create dataset for model training </b>
<br>Get matchups from tournament, get statistics for each team, engineer features comparing team stats

In [14]:
# Get results from each NCAA tournament game
# Each row is a game, with winner team and loser team identified 
ncaa_results_simple = ncaa_results[['Season', 'WTeamID', 'LTeamID']]
ncaa_results_simple.head()

,Season,WTeamID,LTeamID
0,2003,1421,1411
1,2003,1112,1436
2,2003,1113,1272
3,2003,1141,1166
4,2003,1143,1301


In [15]:
# Reformat results dataframe - Randomly reassign Winner/Loser to Team1/Team2 for ML model 
np.random.seed(2)
ncaa_results_simple = ncaa_results_simple.assign(WTeam=np.random.randint(1, 3, ncaa_results_simple.shape[0]))
ncaa_results_simple.head()

,Season,WTeamID,LTeamID,WTeam
0,2003,1421,1411,1
1,2003,1112,1436,2
2,2003,1113,1272,2
3,2003,1141,1166,1
4,2003,1143,1301,1


In [16]:
# Rename WTeamID/LTeamID columns to Team1/Team2
temp1 = ncaa_results_simple.query('WTeam==1')
temp1 = temp1.rename(columns={'WTeamID':'Team1', 'LTeamID': 'Team2'})

temp2 = ncaa_results_simple.query('WTeam==2')
temp2 = temp2.rename(columns={'WTeamID':'Team2', 'LTeamID': 'Team1'})

games_rand = pd.concat([temp1,temp2],axis=0)
games_rand.sort_index(inplace=True)
games_rand.head() # Result = All NCAA games since 2002-2003 with target column

,Season,Team1,Team2,WTeam
0,2003,1421,1411,1
1,2003,1436,1112,2
2,2003,1272,1113,2
3,2003,1141,1166,1
4,2003,1143,1301,1


<b> Combine historical tournament matchups with season stats </b>

In [17]:
# Features to find difference between teams
stat_features = ['OppScore', 'Score','OppFGM','OppFGA', 'OppFGM3', 'OppFGA3','OppFTM', 'OppFTA', 'OppOR', 'OppDR',
 'OppAst', 'OppTO', 'OppStl', 'OppBlk', 'OppPF', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl',
 'Blk', 'PF', 'FGpct', 'FG3pct', 'FTpct']

In [18]:
# Add Team 1's regular season summary stats
merged = pd.merge(games_rand, all_stats[stat_features], how='left', left_on=['Season','Team1'], right_index=True)
team1_stats = ['Team1'+x for x in stat_features]
merged.rename(columns=dict(zip(stat_features, team1_stats)), inplace=True)

# Add Team 2's regular season summary stats
merged = pd.merge(merged, all_stats[stat_features], how='left', left_on=['Season','Team2'], right_index=True)
team2_stats = ['Team2'+x for x in stat_features]
merged.rename(columns=dict(zip(stat_features, team2_stats)), inplace=True)

merged.head()

,Season,Team1,Team2,WTeam,Team1OppScore,Team1Score,Team1OppFGM,Team1OppFGA,Team1OppFGM3,Team1OppFGA3,...,Team2OR,Team2DR,Team2Ast,Team2TO,Team2Stl,Team2Blk,Team2PF,Team2FGpct,Team2FG3pct,Team2FTpct
0,2003,1421,1411,1,78.448276,71.206897,27.793103,60.965517,7.620690,20.758621,...,13.166667,24.800000,14.200000,15.233333,6.433333,2.233333,18.300000,0.447527,0.320721,0.619952
1,2003,1436,1112,2,63.137931,67.793103,22.758621,55.068966,7.068966,21.448276,...,15.178571,27.642857,17.642857,14.785714,8.464286,4.214286,17.750000,0.461413,0.350534,0.701429
2,2003,1272,1113,2,65.827586,74.517241,23.275862,57.862069,5.896552,18.310345,...,13.689655,23.310345,15.551724,14.000000,5.206897,4.241379,19.413793,0.478182,0.317808,0.669737
3,2003,1141,1166,1,73.241379,79.344828,26.344828,57.931034,4.965517,13.896552,...,10.878788,23.181818,16.818182,13.363636,8.393939,4.454545,17.272727,0.499473,0.389053,0.692890
4,2003,1143,1301,1,69.758621,74.482759,25.379310,58.793103,7.103448,21.448276,...,9.733333,22.033333,14.666667,14.200000,7.766667,3.066667,18.666667,0.456250,0.354074,0.770358


<b> Engineer features for the difference in each team's aggregated stats </b>

In [19]:
# Create features representing the difference in Team1, Team2 regular season stat avgs
for x in stat_features:
    merged[x+'_diff'] = merged['Team1'+x] - merged['Team2'+x]

In [20]:
# Features to find difference between teams
diff_stats = [x+'_diff' for x in stat_features]
diff_stats.append('WTeam')

In [21]:
model = merged[diff_stats]
model.head()

,OppScore_diff,Score_diff,OppFGM_diff,OppFGA_diff,OppFGM3_diff,OppFGA3_diff,OppFTM_diff,OppFTA_diff,OppOR_diff,OppDR_diff,...,DR_diff,Ast_diff,TO_diff,Stl_diff,Blk_diff,PF_diff,FGpct_diff,FG3pct_diff,FTpct_diff,WTeam
0,7.614943,-1.593103,2.126437,0.565517,0.087356,-2.408046,3.274713,3.818391,1.790805,-0.039080,...,-1.627586,-1.165517,0.973563,0.635632,0.766667,0.803448,-0.018262,0.039433,0.142815,1
1,-7.112069,-17.421182,-3.598522,-9.609606,0.747537,1.483990,-0.662562,-1.955665,-3.520936,-1.423645,...,-1.918719,-3.435961,-0.716749,-1.602217,-1.248768,-1.853448,-0.016969,-0.009777,-0.043580,2
2,-3.344828,-1.448276,-1.517241,2.206897,0.448276,2.413793,-0.758621,0.206897,1.344828,3.034483,...,2.655172,1.068966,-0.206897,2.172414,0.827586,-0.655172,-0.040251,0.030989,-0.016122,2
3,8.908046,0.102403,2.466040,2.597701,0.086729,-0.406479,3.889237,5.268548,1.180773,-2.467085,...,0.094044,-1.197492,4.877743,-1.290491,-0.454545,3.692790,0.005763,-0.008284,0.072864,1
4,1.758621,2.082759,1.945977,5.659770,1.370115,4.448276,-3.503448,-3.751724,0.639080,1.566667,...,2.345977,1.333333,-0.027586,-1.214943,-0.273563,-1.563218,0.009399,0.022444,-0.084846,1


In [22]:
# Save model DF
model.to_pickle("modelData.pkl")